In [11]:
import sys
from pathlib import Path
ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))

from setup.utils import load_world_sequences, load_model
from decoding_analysis.decoder_helpers import run_label_decoder, run_tuple_decoder, collect_hidden_states
from decoding_analysis.tuple_helpers import setup_tuple
import torch
from setup.model import GP_model


In [12]:
layers = [0, 1, 2]
world_seqs = load_world_sequences("../paper_data/sequences/pentagon_worlds.pkl", n_worlds=500)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = GP_model(tokens_size=26, directions_size=2,
                   embedding_size=200, hidden_size=512, output_size=26,
                   n_layers=3, layer_norm=False, dropout=0.0)
model = load_model(model, device, "../paper_data/model")
batch_tokens, batch_hidden = collect_hidden_states(model=model, world_seqs=world_seqs, device=device, layers=layers)
min_t = 35


loading checkpoint ../paper_data/model/final_epoch_79.pth
start collection


Extracting hidden (batched): 100%|██████████| 1/1 [00:02<00:00,  2.52s/it]

states collected


In [ ]:

run_label_decoder(
    models={"model": model},
    layers=layers,
    offsets=[0, 1, 2],           
    batch_hidden=batch_hidden,
    world_seqs=world_seqs,
    cv_folds=5, 
    min_t=min_t
)



[INFO] Total jobs: 9


Label decoder:   0%|                                                          | 0/9 [00:00<?, ?it/s]

In [ ]:
# decode position from tuple worlds 


mode = "position"
pos_id, _ = setup_tuple(mode=mode)
position_offsets = [(0,0),(1,1), (1,0), (0,1)]

run_tuple_decoder(
    models={"model": model},
    layers=layers,
    offsets=position_offsets,
    batch_hidden=batch_hidden,
    world_seqs=world_seqs,
    cv_folds=5, 
    pos_id=pos_id,
    mode=mode, 
    min_t=min_t)


In [ ]:
# Run tuple decoders
tuples_dict, _ = setup_tuple()
mode = "tuple"
tuple_offsets = [(0,0),(1,1), (1,0), (0,1)]

run_tuple_decoder(
    models={"model": model},
    layers=layers,
    offsets=tuple_offsets,
    batch_hidden=batch_hidden,
    world_seqs=world_seqs,
    tuples_dict=tuples_dict,
    cv_folds=5,
    mode=mode, 
    min_t=min_t)